# **CHAPTER 1. Pangenome analysis**

**Install conda env and activate it**

```
conda env create -f panacota.yaml
```

```
conda activate panacota
```

## **Part 1: Pangenome building**

Make a directory to store the data

In [11]:
! mkdir data/

Download sequences from `RefSeq` database that match the query `"Salmonella phage" AND "complete genome"`

In [ ]:
! esearch -db nucleotide \
    -query '"Salmonella phage" AND "complete genome" AND srcdb_refseq[PROP] ' \
    | efetch -format fasta > data/salmonella_raw.fasta

In the `data/salmonella_raw.fasta` there are many strange sequences like `>NZ_CP019649.1 Salmonella enterica subsp. enterica serovar Typhimurium var. monophasic 4,5,12:i:- strain TW-Stm6 chromosome, complete genome` etc.<br>
Eventhough the query to `RefSeq` database was clear there are still a lot of junk in `.fasta` file.<br>
Let's remove it!<br>
The script below will leave only sequences that has `Salmonella phage` and `complete genome` in their descriptions.

In [ ]:
%run scripts/filter_seqs.py data/salmonella_raw.fasta data/salmonella_251_refseq.fasta

Now it's time to separate sequences!<br>
The script below will take the `.fasta` file with 251 sequences for the input and will store seaparate sequences in the output directory.

In [1]:
%run scripts/sepfasta.py data/salmonella_251_refseq.fasta Salmonella_phages

Separated fasta files are written to Salmonella_phages directory


Create the `listFile` file which will have the names of each file<br>
It is required to run `PanACoTA`

In [ ]:
! ls Salmonella_phages/ > data/listFile

Annotate each genome of `Salmonella phage` that we took in the analysis

In [ ]:
! PanACoTA annotate -d Salmonella_phages/ -r Annotation -n SaPh -l data/listFile --threads 24

Build the pangenome and set the parameter of minimum identity to 80%

In [ ]:
! PanACoTA pangenome -l Annotation/LSTINFO-.lst -n SaPh -d Annotation/Proteins/ -o Pangenome -i 0.8

Now proceed to `02_Pangenome_vis_journal.R` to investigate the structure of the pangenome!<br>
And get back here after then!

## **Part 2: Getting the most commonly shared genes**

So, we must get the genes that are shared in 71 genomes out of 251<br>
Let's calculate the percentage!

In [3]:
round(71 / 251 * 100)

28

Find genes that has 80% identity in at least 28% of genomes

In [ ]:
! PanACoTA corepers -p Pangenome/PanGenome-SaPh.All.prt-clust-0.8-mode1.lst -o Coregenome -t 0.28

Filter `Annotation/LSTINFO-.lst` file and leave there only entries that persist in `Coregenome` output

In [25]:
%run scripts/process_LSTINFO.py Coregenome/PersGenome_PanGenome-SaPh.All.prt-clust-0.8-mode1.lst-all_0.28.lst Annotation/LSTINFO-.lst Annotation/filtered_LSTINFO-.lst

Filtered file created: Annotation/filtered_LSTINFO-.lst


Perform a Multiple Sequences Alignment of that one gene that was identified to have 80% identity in 28% of genomes!

In [ ]:
! PanACoTA align -c Coregenome/PersGenome_PanGenome-SaPh.All.prt-clust-0.8-mode1.lst-all_0.28.lst -l Annotation/filtered_LSTINFO-.lst -n SaPh -d Annotation/ -o Alignment

After that we performed manual BLAST search and decided to design primers for gene family #414

## **Part 3: Phylogenetics 414**

Now we must rename sequences in MSA file from `>SaPh.1224.00003.0001b_00001 420 NA | hypothetical protein | NA | NA | NA` to `>NC_006940.2` etc.

In [5]:
import csv

In [6]:
# Step 1: Load the TSV file into a dictionary
tsv_file = 'Annotation/filtered_LSTINFO-.lst'
name_mapping = {}

with open(tsv_file, 'r') as file:
    reader = csv.DictReader(file, delimiter='\t')
    for row in reader:
        # Mapping the gembase_name to orig_name without the .fasta extension
        gembase_name = row['gembase_name']
        orig_name = row['orig_name'].replace('.fasta', '')  # Remove .fasta extension
        name_mapping[gembase_name] = orig_name

# Step 2: Process the FASTA file and rename headers
fasta_file = 'Alignment/Align-SaPh/SaPh-mafft-align.414.aln'
output_fasta_file = 'Alignment/Align-SaPh/renamed_SaPh-mafft-align.414.aln'

with open(fasta_file, 'r') as infile, open(output_fasta_file, 'w') as outfile:
    lines = infile.readlines()
    
    for line in lines:
        if line.startswith('>'):
            # Extract the gembase_name from the FASTA header (before the first 3rd dot)
            parts = line.split(' ')[0][1:]  # Extract the part before the first space and remove '>'
            gembase_name = '.'.join(parts.split('.')[:3])
            
            # Look up the corresponding NC_XXX.2 name from the dictionary
            if gembase_name in name_mapping:
                new_header = f">{name_mapping[gembase_name]}\n"
                outfile.write(new_header)
            else:
                # If not found in the mapping, keep the original header
                outfile.write(line)
        else:
            # Write sequence lines as they are
            outfile.write(line)

print("FASTA file has been renamed successfully.")

FASTA file has been renamed successfully.


Launch `Model-Finder` to find the best substitution model

In [5]:
! mkdir phylogenetics/
! mkdir phylogenetics/model-finder/

In [ ]:
! iqtree2 -m MFP -s Alignment/Align-SaPh/renamed_SaPh-mafft-align.414.aln --prefix phylogenetics/model-finder/tree_MF2 -T AUTO -redo

In [9]:
! head -42 model-finder/tree_MF2.iqtree | tail -6

Best-fit model according to BIC: Q.plant+G4

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
Q.plant+G4        -1033.401    2238.801 -   0.0143    2526.571 +    0.535    2491.166 +    0.355


Laucnh `IQ-TREE` to build phylogenetic tree

In [ ]:
! mkdir phylogenetics/tree/

In [ ]:
! iqtree2 -s Alignment/Align-SaPh/renamed_SaPh-mafft-align.414.aln -m Q.plant+G4 -pre phylogenetics/tree/tree_ufb -bb 1000 -nt AUTO

Now parse `Annotation/filtered_LSTINFO-.lst` file to leave only `orig_name` column which will be `accession_numbers.txt` file

In [14]:
import csv

# Input and output file names
tsv_file = 'Annotation/filtered_LSTINFO-.lst'
output_txt_file = 'data/accession_numbers.txt'

# Open the TSV file and process it
with open(tsv_file, 'r') as infile, open(output_txt_file, 'w') as outfile:
    reader = csv.DictReader(infile, delimiter='\t')
    
    # Write the header to the output txt file
    #outfile.write("orig_name\n")
    
    # Process each row and write the modified orig_name (without ".fasta")
    for row in reader:
        orig_name = row['orig_name'].replace('.fasta', '')  # Remove ".fasta" extension
        outfile.write(f"{orig_name}\n")

print("Processing complete. The new file has been created.")

Processing complete. The new file has been created.


Install `phyloki`

In [ ]:
! wget https://raw.githubusercontent.com/iliapopov17/phyloki/refs/heads/v0.525/phyloki.py

Import `phyloki`

In [1]:
from phyloki import *

Make a directory to store metadata

In [ ]:
! mkdir metadata/

Fetch metadata!

In [2]:
fetch_metadata('iljapopov17@gmail.com', 'data/accession_numbers.txt', 'metadata/raw_metadata.tsv')

The request has been fulfilled.
File saved to metadata/raw_metadata.tsv


Clean `Year` column to leave only year (withoud the full date)!

In [3]:
def clean_year(input_file, output_file):
    """
    Clean the 'Year' column in a metadata .tsv file to extract only the last 4 digits.

    Args:
        input_file (str): Path to the input .tsv file.
        output_file (str): Path to save the cleaned .tsv file.
    """
    # Load the .tsv file into a DataFrame
    df = pd.read_csv(input_file, sep="\t")

    # Extract the last 4 digits of the 'Year' column
    df['Year'] = df['Year'].apply(lambda x: str(x)[-4:] if pd.notnull(x) else 'ND')

    # Save the updated DataFrame to a new .tsv file
    df.to_csv(output_file, sep="\t", index=False)

    print(f"The 'Year' column has been cleaned.\nFile saved to {output_file}")

Run this function!

In [4]:
clean_year('metadata/raw_metadata.tsv', 'metadata/metadata.tsv')

The 'Year' column has been cleaned.
File saved to metadata/metadata.tsv


That's all!<br>
Proceed to the `03_phylo_journal.R` to build phylogenetic tree and visualize MSA!<br>
After that proceed to the `04_DECIPHER_journal.R` to design degenerative PCR primers for that gene!